# 엘라스틱넷(Elastic Net) 회귀
- 규제 선형 모델: MSE를 최소화하고 회귀계수의 크기를 제어함으로써, 다항 회귀에서의 과적합(overfitting)을 방지
- L1 규제와 L2 규제를 결합
    - L1 규제에 따라 가중치값이 급격하게 변하는 것을 완화, 다만 수행 시간이 상대적으로 길다

In [18]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['PRICE'] = boston.target

y_target = df['PRICE']
X_data = df.drop(['PRICE'], axis=1, inplace=False)

def get_linear_reg_eval(model_name, params=None, X_data_n=None, index_n=None, y_target_n=None):
    coeff_df = pd.DataFrame()
    for param in params:
        if model_name == 'Ridge': model = Ridge(alpha=param)
        elif model_name == 'Lasso': model = Lasso(alpha=param)
        elif model_name == 'ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n, scoring='neg_mean_squared_error', cv=5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {} : {}'.format(param, avg_rmse))
        model.fit(X_data_n, y_target_n)
        coeff = pd.Series(data=model.coef_, index=index_n)
        colname = 'alpha:' + str(param)
        coeff_df[colname] = coeff
    return coeff_df

elastic_alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df = get_linear_reg_eval('ElasticNet', params=elastic_alphas, X_data_n=X_data, index_n=X_data.columns, y_target_n=y_target)

alpha 0.07 : 5.54165434734814
alpha 0.1 : 5.52592849629491
alpha 0.5 : 5.466748649445586
alpha 1 : 5.596874445109748
alpha 3 : 6.068121638621164


In [30]:
sort_column = 'alpha:' + str(elastic_alphas[0])
coeff_elastic_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
RM,3.574162,3.414154,1.918419,0.938789,0.000000
CHAS,1.330724,0.979706,0.000000,0.000000,0.000000
RAD,0.278880,0.283443,0.300761,0.289299,0.146846
ZN,0.050107,0.050617,0.052878,0.052136,0.038268
B,0.010122,0.010067,0.009114,0.008320,0.007020
AGE,-0.010116,-0.008276,0.007760,0.020348,0.043446
TAX,-0.014522,-0.014814,-0.016046,-0.016218,-0.011417
INDUS,-0.044855,-0.042719,-0.023252,-0.000000,-0.000000
CRIM,-0.099468,-0.099213,-0.089070,-0.073577,-0.019058
NOX,-0.175072,-0.000000,-0.000000,-0.000000,-0.000000
